<a href="https://colab.research.google.com/github/zyang63/Die_casting_ejection/blob/main/Copy_of_draft_angle_marking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install trimesh
!pip install bpy
!pip install numpy pillow
!apt-get install blender

In [2]:
import os
import bpy
import math
import numpy as np
from PIL import Image, ImageDraw
import trimesh as t
from trimesh.voxel import creation
import cv2

In [3]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
#@title #File Entry { display-mode: "form"}
#@markdown User can choose to upload the file to colab directly or select the google file upload button at the bottom of this form. Also choose the number of elements.

button_pressed = False  # Initialize the variable as False
filename = ""
button = widgets.Button(description="Google upload dialog")
output = widgets.Output()

def on_button_clicked(b):
    global button_pressed  # Access the global variable
    global filename
    with output:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        button_pressed = True  # Set the variable to True when the button is clicked

button.on_click(on_button_clicked)
display(button, output)

Button(description='Google upload dialog', style=ButtonStyle())

Output()

In [4]:
#@title #Element Count { display-mode: "form", run: "auto" }
element_count = 200 #@param {type:"slider", min:10, max:500, step:1}
if not button_pressed:
  filename = "/content/valve_body (1).STL" #@param {type:"string"}
geometry = t.load_mesh(filename)
#geometry = t.load_mesh("/content/ball.stl")
voxel_size = geometry.extents.max()/element_count
print("Element size is in units from stl file ",voxel_size, " per cell")

Element size is in units from stl file  1.75  per cell


In [5]:
rotation_angle = 90
Mu = 0.6
Max_scale_value = 0.05

# Remesh and output OBJ file

In [6]:
blender_script = f"""
import bpy

bpy.ops.wm.read_factory_settings(use_empty=True)

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath= "{filename}", global_scale=0.1,)

bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')
selected_object = bpy.context.object



# Export the object as an OBJ file
for obj in bpy.context.selected_objects:
    if obj.type == "MESH":
        bpy.ops.export_scene.obj(filepath="/content/remeshed_uv.obj", use_triangles=False, use_materials=False)

file_path = "/content/meshed_geometry.blend"
# Save the file
bpy.ops.wm.save_as_mainfile(filepath=file_path)"""
script_filename = "/content/blender_script.py"
with open(script_filename, "w") as script_file:
    script_file.write(blender_script)

# Run Blender in headless mode with the script
!blender -b -P {script_filename}

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: scene view "Filmic" not found, setting default "Standard".
Import finished in 0.4191 sec.
    (  0.0003 sec |   0.0000 sec) OBJ Export path: '/content/remeshed_uv.obj'
          ( 15.8451 sec |  15.8446 sec) Finished writing geometry of 'valve_body (1)'.
      ( 15.8452 sec |  15.8450 sec) Finished exporting geometry, now exporting materials
      ( 15.8453 sec |  15.8450 sec) OBJ Export Finished
Progress: 100.00%

Info: Total files 0 | Changed 0 | Failed 0
Info: Saved "meshed_geometry.blend"
Info: Saved "meshed_geometry.blend"

Blender quit


In [7]:
obj_file_path = '/content/remeshed_uv.obj'

# Ejection force and Friction coefficient

In [8]:
def force_coefficient_calculation(selected_axis_data, obj_file_path, Mu):
    vertices, faces, vt_coordinates = [], [], []
    vt_faces, abs_F_list, tangents, abs_tangents = [], [], [], []
    direction = np.array(selected_axis_data)
    with open(obj_file_path, 'r') as obj_file:
        for line in obj_file:
            parts = line.split()
            if parts[0] == 'v':
                vertices.append(list(map(float, parts[1:3])) + [float(parts[3]) if len(parts) > 3 else 0.0])
            elif parts[0] == 'f':
                face, vt_face = zip(*((int(x.split('/')[0]) - 1, int(x.split('/')[1]) - 1) for x in parts[1:]))
                faces.append(face)
                vt_faces.append(vt_face)
            elif parts[0] == 'vt':
                vt_coordinates.append(list(map(float, parts[1:])))
    for face in faces:
        normal = np.cross(*(np.array(vertices[face[i]]) - np.array(vertices[face[0]]) for i in (1, 2)))
        normal /= np.linalg.norm(normal)
        cosine = np.dot(normal, direction) / np.linalg.norm(direction)
        sine = np.sqrt(1 - cosine**2)
        abs_F_list.append(Mu * abs(sine) - abs(cosine))
        tangents.append(cosine / sine)
        abs_tangents.append(abs(cosine / sine))
    return vt_coordinates, vt_faces, abs_F_list, tangents, abs_tangents

#Direction


In [9]:
def normal_force_rotation(rotation_angle):
    def rotation_matrix(axis, theta):
        theta = np.radians(theta)
        if axis == 'x':
            return np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
        elif axis == 'y':
            return np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'z':
            return np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    unique_rotations = {}
    for theta_x in range(0, 360 + rotation_angle, rotation_angle):
        for theta_y in range(0, 360 + rotation_angle, rotation_angle):
            for theta_z in range(0, 360 + rotation_angle, rotation_angle):
                rotated_vector = np.dot(rotation_matrix('x', theta_x),
                                        np.dot(rotation_matrix('y', theta_y),
                                               np.dot(rotation_matrix('z', theta_z), np.array([0, 1, 0]))))
                rotated_vector = tuple(np.where(np.abs(rotated_vector) < 0.001, 0, rotated_vector))
                unique_rotations.setdefault(rotated_vector, (theta_x, theta_y, theta_z))
    unique_angles_list = list(unique_rotations.values())
    unique_vectors_list = [list(vec) for vec in unique_rotations.keys()]
    return unique_angles_list, unique_vectors_list

In [10]:
counts = [sum(1 for t in force_coefficient_calculation(direction, obj_file_path, Mu)[3] if t < Mu)
    for direction in normal_force_rotation(rotation_angle)[1]]
die_separation_direction = normal_force_rotation(rotation_angle)[1][np.argmin(counts)]

<ipython-input-8-c5466d3dcf86>:24: RuntimeWarning: divide by zero encountered in scalar divide
  tangents.append(cosine / sine)
<ipython-input-8-c5466d3dcf86>:25: RuntimeWarning: divide by zero encountered in scalar divide
  abs_tangents.append(abs(cosine / sine))


In [11]:
print(counts)
print(corresponding_row)

[247500, 258378, 244191, 258289, 237174, 237009]
[0.0, 0.0, -1.0]


In [12]:
vt_coordinates, vt_faces, abs_F_list, tangents, abs_tangents = force_coefficient_calculation(corresponding_row, obj_file_path, Mu)

<ipython-input-8-c5466d3dcf86>:24: RuntimeWarning: divide by zero encountered in scalar divide
  tangents.append(cosine / sine)
<ipython-input-8-c5466d3dcf86>:25: RuntimeWarning: divide by zero encountered in scalar divide
  abs_tangents.append(abs(cosine / sine))


In [13]:
abs_F_array = np.array(abs_F_list)
max_abs_F = abs_F_array.max()
min_abs_F = abs_F_array.min()
tangents_array = np.array(tangents)
tangent_filtered_array = tangents_array[~np.isnan(tangents_array) & np.isfinite(tangents_array)]
abs_tangents_array = np.array(abs_tangents)
abs_tangent_filtered_array = abs_tangents_array[~np.isnan(abs_tangents_array) & np.isfinite(abs_tangents_array)]

In [14]:
print(abs_F_array)

[ 0.6         0.6         0.6        ...  0.6        -0.9108459
 -0.41242985]


# Texture marked

In [105]:
# Determine the image size.
image_size = 1080

color_bins = [(i, 0, 256-i) for i in range(0, 256, 26)]

# Create a black image.
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)

# Scale the VT coordinates to fit the 1080x1080 image size.
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates
]

# Loop through the faces and apply red or blue coloring based on cosines.
for i, (vt_face, abs_F) in enumerate(zip(vt_faces, abs_F_list)):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        #print(vt_indices)
        fraction = (abs_F - min_abs_F) / (max_abs_F - min_abs_F)
        index = int(fraction * 10)
        index = max(0, min(index, 9))  # Ensure the index is within the valid range

    # Use the selected color from the color_bins list
        color = color_bins[index]
        draw.polygon(
            vt_indices,
            outline=color,
            fill=color,
        )

# Save the marked image.
image.save('marked_image_abs_F.png')


In [15]:
# Determine the image size.
image_size = 1080

color_bins = [(256-i, 0, i) for i in range(0, 256, 26)]

# Create a black image.
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)

# Scale the VT coordinates to fit the 1080x1080 image size.
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates
]

# Loop through the faces and apply red or blue coloring based on cosines.
for i, (vt_face, abs_tangent) in enumerate(zip(vt_faces ,abs_tangents)):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        if abs_tangent != float('inf') and 0 <= abs_tangent <= Max_scale_value:
          fraction = (abs_tangent - 0) / Max_scale_value

    # Map fraction to one of the 10 color bins
          index = int(fraction * 10)
          index = max(0, min(index, 9))  # Ensure the index is within the valid range

    # Use the selected color from the color_bins list
          color = color_bins[index]
        elif abs_tangent != float('inf') and  abs_tangent > Mu:
            color = (125, 125, 125)
        elif abs_tangent != float('inf') and  Max_scale_value < abs_tangent <= Mu:
            color = (0, 0, 255)
        elif abs_tangent == float('inf'):
            color = (125, 125, 125)
        # Draw a red or blue polygon on the image.
        draw.polygon(
            vt_indices,
            outline=color,
            fill=color,
        )

# Save the marked image.
image.save('marked_image_abs_tangents.png')

In [17]:
image_size = 1080
color_bins = [(256 - i, 0, i) for i in range(0, 256, 26)]
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)
scaled_vt_coordinates = [(vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates]
for vt_face, abs_tangent in zip(vt_faces, abs_tangents):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        if abs_tangent == float('inf') or abs_tangent > Mu:
            color = (125, 125, 125)
        elif abs_tangent > Max_scale_value:
            color = (0, 0, 255)
        else:
            fraction = max(0, min((abs_tangent / Max_scale_value), 1))
            color = color_bins[int(fraction * 10)]
image.save('marked_image_abs_tangents.png')

In [ ]:
# Determine the image size.
image_size = 1080

color_bins_positive = [(256-i, 0, 0) for i in range(0, 256, 26)]
color_bins_negative = [(0, 0, 256-i) for i in range(0, 256, 26)]

# Create a black image.
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)

# Scale the VT coordinates to fit the 1080x1080 image size.
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates
]

# Loop through the faces and apply red or blue coloring based on cosines.
for i, (vt_face, tangent) in enumerate(zip(vt_faces ,tangents)):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        if tangent != float('inf') and 0 < tangent <= Max_scale_value:
          fraction = (tangent - 0) / Max_scale_value

    # Map fraction to one of the 10 color bins
          index = int(fraction * 10)
          index = max(0, min(index, 9))  # Ensure the index is within the valid range

    # Use the selected color from the color_bins list
          color = color_bins_positive[index]
        elif tangent != float('inf') and (-Max_scale_value) <= tangent < 0:
          fraction = (tangent -0) / (-Max_scale_value)

    # Map fraction to one of the 10 color bins
          index_1 = int(fraction * 10)
          index_1 = max(0, min(index, 9))  # Ensure the index is within the valid range
          color = color_bins_negative[index]
        elif tangent != float('inf') and  tangent > Mu:
            color = (125, 125, 125)
        elif tangent != float('inf') and  tangent < (-Mu):
            color = (125, 125, 125)
        elif tangent != float('inf') and  Max_scale_value < tangent <= Mu:
            color = (255, 0, 0)
        elif tangent != float('inf') and  (-Mu) <= tangent < (-Max_scale_value):
            color = (0, 0, 255)
        elif tangent == 0:
            color = (255, 255, 255)
        elif tangent == float('inf'):
            color = (125, 125, 125)
        # Draw a red or blue polygon on the image.
        draw.polygon(
            vt_indices,
            outline=color,
            fill=color,
        )

# Save the marked image.
image.save('marked_image_tangents.png')

In [ ]:
# Determine the image size.
image_size = 1080

# Create a black image.
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)

# Scale the VT coordinates to fit the 1080x1080 image size.
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates
]

# Loop through the faces and apply red or blue coloring based on cosines.
for i, (vt_face, tangent) in enumerate(zip(vt_faces ,tangents)):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        if tangent != float('inf') and 0 < tangent <= Mu:
          color = (255,0,0)
        elif tangent != float('inf') and  tangent > Mu:
            color = (0, 0, 0)
        elif tangent != float('inf') and  tangent < (-Mu):
            color = (0, 0, 0)
        elif tangent != float('inf') and  (-Mu) <= tangent < 0:
            color = (0, 0, 255)
        elif tangent == 0:
            color = (255, 255, 255)
        elif tangent == float('inf'):
            color = (0, 0, 0)
        # Draw a red or blue polygon on the image.
        draw.polygon(
            vt_indices,
            outline=color,
            fill=color,
        )

# Save the marked image.
image.save('marked_sum_tangents.png')

In [ ]:
# For example, you can upload an image to Colab and use its path
image_path = '/content/marked_sum_tangents.png'
image = cv2.imread(image_path)

# Convert the image to RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define the color values for red and blue
red_color = np.array([255, 0, 0])
blue_color = np.array([0, 0, 255])

# Count the number of pixels matching the red and blue colors
red_pixel_count = np.sum(np.all(image_rgb == red_color, axis=-1))
blue_pixel_count = np.sum(np.all(image_rgb == blue_color, axis=-1))
print("Number of red pixels:", red_pixel_count)
print("Number of blue pixels:", blue_pixel_count)
# Compare the number of red and blue pixels and print "top" if red is larger
if red_pixel_count > blue_pixel_count:
    print("Top need more force")
else:
    print("Bottom need more force")


Number of red pixels: 153088
Number of blue pixels: 232989
Bottom need more force


# Visualization

In [106]:
bpy.ops.wm.open_mainfile(filepath="/content/meshed_geometry.blend")
material = bpy.data.materials.new(name="MyMaterial")

# Create a new shader node tree for the material
material.use_nodes = True
tree = material.node_tree
nodes = tree.nodes

# Clear existing nodes
for node in nodes:
    nodes.remove(node)

# Create a new Principled BSDF node
bsdf_node = nodes.new(type='ShaderNodeBsdfPrincipled')
bsdf_node.location = (0, 0)  # Position the node in the node editor

# Create a new Image Texture node
texture_node = nodes.new(type='ShaderNodeTexImage')
texture_node.location = (-200, 0)  # Position the node in the node editor

# Set the path to the image texture
texture_node.image = bpy.data.images.load("/content/marked_image_abs_F.png")

# Connect the Image Texture node to the Base Color input of the Principled BSDF node
base_color_link = tree.links.new(texture_node.outputs['Color'], bsdf_node.inputs['Base Color'])

# Create a new Material Output node
output_node = nodes.new(type='ShaderNodeOutputMaterial')
output_node.location = (200, 0)  # Position the node in the node editor

# Connect the Principled BSDF node to the Surface input of the Material Output node
surface_link = tree.links.new(bsdf_node.outputs['BSDF'], output_node.inputs['Surface'])

# Assign the material to the active object
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath="/content/colored_F_geometry.glb")

06:20:03 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
06:20:03 | INFO: Starting glTF 2.0 export
06:20:05 | INFO: Extracting primitive: cylinder_2_degree
06:20:12 | INFO: Primitives created: 1
06:20:13 | INFO: Finished glTF 2.0 export in 10.480602741241455 s



{'FINISHED'}

In [16]:
bpy.ops.wm.open_mainfile(filepath="/content/meshed_geometry.blend")
material = bpy.data.materials.new(name="MyMaterial")

# Create a new shader node tree for the material
material.use_nodes = True
tree = material.node_tree
nodes = tree.nodes

# Clear existing nodes
for node in nodes:
    nodes.remove(node)

# Create a new Principled BSDF node
bsdf_node = nodes.new(type='ShaderNodeBsdfPrincipled')
bsdf_node.location = (0, 0)  # Position the node in the node editor

# Create a new Image Texture node
texture_node = nodes.new(type='ShaderNodeTexImage')
texture_node.location = (-200, 0)  # Position the node in the node editor

# Set the path to the image texture
texture_node.image = bpy.data.images.load("/content/marked_image_abs_tangents.png")

# Connect the Image Texture node to the Base Color input of the Principled BSDF node
base_color_link = tree.links.new(texture_node.outputs['Color'], bsdf_node.inputs['Base Color'])

# Create a new Material Output node
output_node = nodes.new(type='ShaderNodeOutputMaterial')
output_node.location = (200, 0)  # Position the node in the node editor

# Connect the Principled BSDF node to the Surface input of the Material Output node
surface_link = tree.links.new(bsdf_node.outputs['BSDF'], output_node.inputs['Surface'])

# Assign the material to the active object
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath="/content/colored_abs_tangent_geometry.glb")

03:59:35 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
03:59:36 | INFO: Starting glTF 2.0 export
03:59:36 | INFO: Extracting primitive: valve_body (1)
03:59:39 | INFO: Primitives created: 1
03:59:39 | INFO: Finished glTF 2.0 export in 3.825425863265991 s



{'FINISHED'}

In [18]:
bpy.ops.wm.open_mainfile(filepath="/content/meshed_geometry.blend")
material = bpy.data.materials.new(name="MyMaterial")
material.use_nodes = True
nodes = material.node_tree.nodes
links = material.node_tree.links
nodes.clear()
bsdf_node = nodes.new(type='ShaderNodeBsdfPrincipled')
texture_node = nodes.new(type='ShaderNodeTexImage')
output_node = nodes.new(type='ShaderNodeOutputMaterial')
texture_node.image = bpy.data.images.load("/content/marked_image_abs_tangents.png")
bsdf_node.location, texture_node.location, output_node.location = (0, 0), (-200, 0), (200, 0)
links.new(texture_node.outputs['Color'], bsdf_node.inputs['Base Color'])
links.new(bsdf_node.outputs['BSDF'], output_node.inputs['Surface'])
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath="/content/colored_abs_tangent_geometry.glb")

05:22:53 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
05:22:53 | INFO: Starting glTF 2.0 export
05:22:53 | INFO: Extracting primitive: valve_body (1)
05:22:57 | INFO: Primitives created: 1
05:22:57 | INFO: Finished glTF 2.0 export in 3.7976269721984863 s



{'FINISHED'}

In [ ]:
bpy.ops.wm.open_mainfile(filepath="/content/meshed_geometry.blend")
material = bpy.data.materials.new(name="MyMaterial")

# Create a new shader node tree for the material
material.use_nodes = True
tree = material.node_tree
nodes = tree.nodes

# Clear existing nodes
for node in nodes:
    nodes.remove(node)

# Create a new Principled BSDF node
bsdf_node = nodes.new(type='ShaderNodeBsdfPrincipled')
bsdf_node.location = (0, 0)  # Position the node in the node editor

# Create a new Image Texture node
texture_node = nodes.new(type='ShaderNodeTexImage')
texture_node.location = (-200, 0)  # Position the node in the node editor

# Set the path to the image texture
texture_node.image = bpy.data.images.load("/content/marked_image_tangents.png")

# Connect the Image Texture node to the Base Color input of the Principled BSDF node
base_color_link = tree.links.new(texture_node.outputs['Color'], bsdf_node.inputs['Base Color'])

# Create a new Material Output node
output_node = nodes.new(type='ShaderNodeOutputMaterial')
output_node.location = (200, 0)  # Position the node in the node editor

# Connect the Principled BSDF node to the Surface input of the Material Output node
surface_link = tree.links.new(bsdf_node.outputs['BSDF'], output_node.inputs['Surface'])

# Assign the material to the active object
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath="/content/colored_tangent_geometry.glb")

17:55:51 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
17:55:51 | INFO: Starting glTF 2.0 export
17:55:51 | INFO: Extracting primitive: Casting_Shot - 8M0111257-1
17:55:53 | INFO: Primitives created: 1
17:55:53 | INFO: Finished glTF 2.0 export in 1.3204104900360107 s



{'FINISHED'}

In [ ]:
files_to_delete = [
    "/content/meshed_geometry.blend",
    "/content/remeshed_uv.obj",
    "/content/marked_image_abs_F.png",
    "/content/marked_image_tangents.png",
    "/content/marked_image_abs_tangents.png",
    "/content/blender_script.py",
    "/content/marked_sum_tangents.png"
]

for file_path in files_to_delete:
    if os.path.exists(file_path):
        os.remove(file_path)